In [1]:
print("hello")

hello


# dataset

In [ ]:

nu_dataset = NuSceneDataset(**args)

input_dict = nu_dataset.__getitem__(idx) # 获取元信息（文件路径、真值标注）
load_images(input_dict, **args)
load_points(input_dict, **args)
resize_images(input_dict, **args)
get_gt(input_dict, **args)
others(input_dict, **args)
Collect3D(input_dict, **args)


# model

In [ ]:
class Sparse4dNet(nn.Module):
    def __init__(self, args):
        super(Sparse4dNet, self).__init__()
        pass

    def forward(self, **data):
        if self.training:
            return self.forward_train(**data)
        else:
            return self.forward_test(**data)
    
    def foward_train(self, **data):
        imgs = data['imgs']
        # bs * n_cams * 3 * W * H -> n_levels * [(bs* n_cam) * n_chanels * W/@ * H/@]
        features = self.extract_feat(imgs) 

        anchors, cls = self.head(features, data)

        loss = self.head.loss(anchors, cls, data)

    def extract_feat(self, imgs):
        # bs * n_cams * 3 * W * H -> n_levels * [bs * n_cam * 3 * W/@ * H/@]
        features = self.img_backbone(imgs)
        features = self.img_neck(features)
        return features
    
class Sparse4dHead(nn.Module):
    def __init__(self, args):
        super(Sparse4dHead, self).__init__()
        pass

    def foward(self, features, more_features, data):

        # instance features/anchors
        instance_features = 
        instance_anchors = 
        more_instance_features = 
        more_instance_anchors = 

        # SA
        instance_features = SA(instance_features, query_pos = instance_anchors)

        # residual connect

        # feature aggregation
        keypoints_fix = 
        keypoints_learned = kps(instance_features)
        instance_keypoints, more_instance_keypoints = attach(keypoints_fix, keypoints_learned, instance_anchors)

        features_sampled = sample(instance_keypoints, features, more_instance_keypoints, more_features)
        weights = weights_map(instance_anchors)
        features_sampled = weight_sum(features_sampled, weights) # multi-level multi-view

        weight_depth = get_weight_depth(instance_anchors, instance_features)
        features_sampled = weights_depth * features_sampled

        features_sampled = temporal_fusion(features_sampled)
        features_sampled = keypoints_fusion(features_sampled)

        instance_features = fusion(instance_features, features_sampled)

        # refine
        anchors = anchor_head(features_sampled)
        clses = cls_head(features_sampled)

        # loops-->

        return anchors, clses

    def loss(self, anchors, clses, anchors_gt, clses_gt):
        loss = {}

        # match
        cls_cost = cal_cls_cost(clses, clses_gt) # focal loss - pos_loss- neg_loss // * cls_weight
        reg_cost = cal_reg_cost(anchors, anchors_gt) # L1 distance  // * reg_weight
        cost = cls_cost + reg_cost
        match_result = match(cost)

        # 监督信号
        anchors_target = get_target_anchors(match_result, anchors, anchors_gt)
        clses_target = get_target_clses(match_result, clses)

        loss['cls_loss'] = focal_loss(clses, clses_target, avg_size = @)
        loss['reg_loss'] = smooth_l1(anchors, anchors_target, avg_size = @)

        return loss


model = Model(**args)

